## TP1 d'Apprentissage Statistique 

### Binome:
- NKOUNGHAWE TOMEYUM Rosalie Corine
- LENO Celestine  

### A- Les données 

La fonction qui permet d'engendrer une matrice dont chaque valeur est une observation d'une personne avec 3 caractéristiques 

In [64]:
import random
import numpy as np 

def generer_tableau(n): # n est le nombre de lignes de la matrice, le nombre de colonnes est 3

    # Nous définissons les domaines de chaque observation, dans lesquels nous allons faire la sélection des valeurs 
    # pour former la matrice 

    cheveux = ['B', 'D']   # Blond, Dark
    hauteur = ['T', 'S']   # Tall, Short
    pays = ['G', 'P']      # Greenland, Poland
    
    tableau = []
    for _ in range(n):  # pour chaque ligne
        ligne = []
        for _ in range(3):  # 3 colonnes
            c = random.choice(cheveux)
            h = random.choice(hauteur)
            p = random.choice(pays)
            ligne.append((c, h, p))  # un triplet
        tableau.append(ligne)
    return tableau

# Générons une matrice un peu plus grande avec 100 lignes 
print(generer_tableau(100))


[[('B', 'T', 'P'), ('B', 'S', 'G'), ('B', 'S', 'P')], [('B', 'S', 'G'), ('B', 'T', 'P'), ('D', 'T', 'G')], [('B', 'T', 'P'), ('B', 'T', 'P'), ('D', 'T', 'P')], [('B', 'T', 'P'), ('B', 'S', 'G'), ('B', 'S', 'P')], [('D', 'T', 'G'), ('B', 'T', 'P'), ('D', 'S', 'G')], [('D', 'S', 'P'), ('D', 'T', 'G'), ('D', 'T', 'P')], [('B', 'T', 'P'), ('D', 'S', 'P'), ('B', 'S', 'G')], [('D', 'T', 'P'), ('B', 'T', 'P'), ('D', 'T', 'P')], [('D', 'T', 'P'), ('D', 'S', 'P'), ('D', 'S', 'G')], [('B', 'S', 'P'), ('B', 'S', 'P'), ('B', 'S', 'P')], [('B', 'S', 'P'), ('D', 'S', 'P'), ('B', 'S', 'G')], [('B', 'S', 'P'), ('B', 'T', 'P'), ('B', 'T', 'G')], [('B', 'T', 'G'), ('D', 'T', 'P'), ('D', 'T', 'P')], [('D', 'S', 'G'), ('B', 'T', 'G'), ('D', 'T', 'G')], [('D', 'T', 'G'), ('B', 'S', 'G'), ('D', 'S', 'P')], [('D', 'S', 'G'), ('D', 'S', 'P'), ('B', 'S', 'P')], [('B', 'S', 'P'), ('B', 'S', 'G'), ('D', 'S', 'P')], [('B', 'T', 'G'), ('B', 'S', 'G'), ('B', 'T', 'P')], [('D', 'S', 'P'), ('D', 'T', 'G'), ('D', 'S',

In [65]:
# La matrice que nous allons utiliser 

# Pour controler l'aléa 
random.seed(42)
data = generer_tableau(100)

### B- Questions : nationalité la plus probable d'un individu grand (T) et qui a les cheveux Blond (B)

Les fonctions que nous allons écrire dans la suite sont par rapport à la matrice que nous avons définie

### C- Maximum a posteriori
Nous allons utiliser deux versions de cette métrique pour conclure sur la question:  
- Une qui se fera avec naive bayes 
- Une autre qui ne tiendra pas compte de navive bayes : se basera uniquement sur les fréquences d'apparution, via la formule des probabilités conditionnelles

In [66]:
## Nous allons donc faire le calcul de probabilité avec plusieurs petites fonctions que nous allons utiliser
# par la suite dans une fonction générale 

##1- Version avec le théorème de bayes

"""
Il s'agit de calculer deux probabilités a posteriori et de prendre le maximum afin de conclure sur la nationalité de l'individu
Les formules sont les suivantes: 

P(G/B,T) = P(G)*P(B,T/G) / P(B,T)
P(P/B,T) = P(P)*P(B,T/P) / P(B,T)

Nous aurons besoin de caluler :
P(G): pg(), 
P(P): pp(), 
P(B,T/G): pbtg(), 
P(B,T/P): pbtp() 
et P(B,T): pbt()

P(G) = nombre de personnes de nationalité G / nombre total d'individus
P(P) = nombre de personnes de nationalité P / nombre total d'individus

P(B,T/G) = P(B/G)*P(T/G) via l'hypothèse d'indépendance de naive bayes
P(B/G) : pbg() = nombre de personnes aux cheveux Blonds / nombre de personnes de nationalité G
P(T/G) : ptg() = nombre de personnes de hauteur T / nombre de personnes de nationalité G

P(B,T/P) = P(B/P)*P(T/P) pareil que précédemment pour la nationalité G
P(B/P) : pbp() = nombre de personnes aux cheveux Blonds / nombre de personnes de nationalité P
P(T/P) : ptp() = nombre de personnes de hauteur T / nombre de personnes de nationalité P

"""


# -------------- Fonctions utilitaires -------------------- #


def pg(data):
    nb_G = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'G':
                nb_G += 1
    return nb_G / nb_Total

def pp(data):
    nb_P = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'P':
                nb_P += 1
    return nb_P / nb_Total

def pbg(data):
    nb_blond_G, nb_G = 0, 0
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'G':
                nb_G += 1
                if indiv[0] == 'B':  # cheveux blonds
                    nb_blond_G += 1
    return nb_blond_G / nb_G if nb_G > 0 else 0

def ptg(data):
    nb_T_G, nb_G = 0, 0
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'G':
                nb_G += 1
                if indiv[1] == 'T':
                    nb_T_G += 1
    return nb_T_G / nb_G if nb_G > 0 else 0

def pbp(data):
    nb_blond_P, nb_P = 0, 0
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'P':
                nb_P += 1
                if indiv[0] == 'B':  # cheveux blonds
                    nb_blond_P += 1
    return nb_blond_P / nb_P if nb_P > 0 else 0

def ptp(data):
    nb_T_P, nb_P = 0, 0
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'P':
                nb_P += 1
                if indiv[1] == 'T':
                    nb_T_P += 1
    return nb_T_P / nb_P if nb_P > 0 else 0

def pbtg(data):
    return pbg(data) * ptg(data)

def pbtp(data):
    return pbp(data) * ptp(data)

def pbt(data):
    # P(B,T) = somme des cas pour G et P
    # Nous appliquons la loi des proba totales, vu qu'on veut les individus 
    # avec des cheveux B et une hauteur T quelque soit la nationalité
    
    return pbtg(data) * pg(data) + pbtp(data) * pp(data)


# -------------------------Fonction générale------------------#


def classer_individu(data):
    # probas a posteriori
    p_G = (pg(data) * pbtg(data)) / pbt(data)
    p_P = (pp(data) * pbtp(data)) / pbt(data)

    if p_G > p_P:
        return f"Nationalité G avec une proba Max de {p_G}"
    elif p_P > p_G:
        return f"Nationalité P avec une proba Max de {p_P}"
    else:
        return "Indécis (égalité)"


In [67]:
classer_individu(data)

'Nationalité P avec une proba Max de 0.510296618174948'

### Conclusion sur la nationalité avec la première approche: 

Nous voyons que la probabilité de la nationalité P est celle trouvé comme plus grande 

## Reprenons la fonction cette fois sans utiliser le théorème de naive bayes

Il sera question maintenant de faire le calcul de proba conditionnelle

P(G/B,T) = P(B,T,G)/P(B,T) un peu comme la proba de l'intersection de G avec (B,T) divisé par P(B,T)

Ici il s'agit de compter le nombre de fois qu'on a un individu B, T, G et diviser par le nombre d'individu total ensuit diviser par le calul du nombre de personnes aux cheveux B et une hauteur T et diviser par le nombre total d'individus 

P(P/B,T): on fait pareil sur la nationalité P, on regarde le max et on conclu sur la nationalité 

In [68]:
# Fonctions utilitaires pour cette deuxième version qui n'utilise pas le théorème de Bayes 

"""
Ici, on calcule directement les probabilités par comptage dans le jeu de données.

Formules (probabilité conditionnelle par fréquence):

P(G / B,T) = P(B,T,G) / P(B,T) 
P(B,T,G) = nombre d'individus (B,T,G) / nombre total d'individus
P(B,T)   = nombre d'individus (B,T,*) / nombre total d'individus

P(P / B,T) = P(B,T,P) / P(B,T) 
P(B,T,P) = nombre d'individus (B,T,P) / nombre total d'individus
"""

def pbtg_v2(data):
    nb_BTG = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne: 
            if indiv[0] == 'B' and indiv[1] == 'T' and indiv[2] == 'G':
                nb_BTG += 1
    return nb_BTG / nb_Total if nb_Total > 0 else 0

def pbtp_v2(data):
    nb_BTP = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne: 
            if indiv[0] == 'B' and indiv[1] == 'T' and indiv[2] == 'P':
                nb_BTP += 1
    return nb_BTP / nb_Total if nb_Total > 0 else 0

def pbt_v2(data):
    nb_BT = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne: 
            if indiv[0] == 'B' and indiv[1] == 'T':  # Nationalité indifférente
                nb_BT += 1
    return nb_BT / nb_Total if nb_Total > 0 else 0

def classer_individu_v2(data):
    p_G = pbtg_v2(data) / pbt_v2(data) if pbt_v2(data) > 0 else 0
    p_P = pbtp_v2(data) / pbt_v2(data) if pbt_v2(data) > 0 else 0
    
    if p_G > p_P:
        return f"Nationalité G avec proba:  (p={p_G:.3f})"
    elif p_P > p_G:
        return f"Nationalité P avec proba: (p={p_P:.3f})"
    else:
        return f"Indécis (égalité, la proba est: p={p_G:.3f})"


In [69]:
classer_individu_v2(data)

'Nationalité P avec proba: (p=0.560)'

### Commentaire:

Avec notre jeu de données, on constate que les deux approches disent que l'individu est de nationalité P bien que nous remarquons que la probabilité avec la seconde approche semble être plus grande 

La différence entre les deux approches est que la première est plus stable, c'est moins probable d'avoir des 0 comme avec la deuxième approche. En effet, P(B,T/G) avec la deuxième approche vu qu'on calcul P(B,T,G), c'est parfois difficile d'avoir cette combinaison dans le jeu de donnée, ce qui n'est pas le cas lorsqu'on suppose l'indépendance entre les variables et qu'on applique la formule P(B/G)*P(T/G)

## D- Estimateur de Maximum de Vraisemblance : MLE

Il est question dans cette partie de trouver la classe qui maximise plutôt cette probabilité : P((B,T)/nationalité)

Donc *P(observation/classe)*:
- P((B,T)/P) : la nationalité P
- P((B,T)/G) : la nationalité G

Nous devons faire ceci en utilisant deux approches:
1. En tenant compte du cadre bayes naïf
P(B,T/G) = P(B/G)*P(T/G) comme donné dans l'énnoncé du TP pareil pour P(B,T/P)
2. Sans tenir compte du cadre bayes naïf

    Ici on fonctionnera par comptage dans notre matrice de données, comme avec l'exemple donné dans l'énnoncé du TP 


In [70]:
pbtg_sans_naif = pbg(data)*ptg(data)

print(pbtg_sans_naif)

0.24319759804841432


In [71]:
pbtp_sans_naif = pbp(data)*ptp(data)

print(pbtp_sans_naif)

0.24025974025974026


#### Fonction plus générale qui prend en entrée les données et les deux paramètres (couleur_cheveux, hauteur)

In [72]:
# Le code pour répondre à la question avec la première approche est le suivant 


# Notre fonction est un peu plus générale, nous allons lui passer les données, la couleur des cheveux B et la hauteur T pour voir 
# la classe qui maximise la vraisemblance 
def mle_naif(data, cheveux, hauteur):
    # Comptage des individus par classe
    # On récupère de manière individuelle les classes présentes dans les données
    classes = set(indiv[2] for ligne in data for indiv in ligne)

    # où on pourra stocker les vraissemblances calculées et retenir la valeur maximale 
    vraisemblances = {}

    # rappel: le triplet (B,T,G) est dans l'ordre des caractéristiques (cheveux, hauteur et nationalité)
    
    for c in classes:
        nb_c = sum(indiv[2] == c for ligne in data for indiv in ligne)
        nb_cheveux_c = sum(indiv[2] == c and indiv[0] == cheveux for ligne in data for indiv in ligne)
        nb_hauteur_c = sum(indiv[2] == c and indiv[1] == hauteur for ligne in data for indiv in ligne)
        
        p_cheveux = nb_cheveux_c / nb_c if nb_c > 0 else 0
        p_hauteur  = nb_hauteur_c / nb_c if nb_c > 0 else 0
        
        vraisemblances[c] = p_cheveux * p_hauteur
    
    # Choisir la classe qui maximise P(x|c)
    best_class = max(vraisemblances, key=vraisemblances.get)
    return best_class, vraisemblances


In [73]:
## Test de notre fonction pour répondre à la question

print(f"La distribution de vraisemblance entre les classes est: {mle_naif(data=data, cheveux='B', hauteur='T')[1]} \nLa nationalité finale est: {mle_naif(data=data, cheveux='B', hauteur='T')[0]}")

La distribution de vraisemblance entre les classes est: {'G': 0.24319759804841432, 'P': 0.24025974025974026} 
La nationalité finale est: G


In [74]:
## Si on utilisait nos fonctions définies plus haut, voici comment serait simplement le code 

def mle_naif_v2(data):

    # on calcul P(B,T/G) = P(B/G)*P(T/G) on l'appellera p1
    p1 = pbg(data)*ptg(data)

    # On calcul ensuite P(B,T/P) = P(B/P) * P(T/P) on l'appellera p2
    p2 = pbp(data)*ptp(data)
    
    # comparaison et choix de la nationalité 
    if p1 > p2:
        print(f"L'individu (B,T) est de nationalité G avec une vraisemblance de {p1}")
    else:
        print(f"L'individu (B,T) est de nationalité P avec une vraisemblance de {p2}")

In [75]:
## Application 
mle_naif_v2(data)

L'individu (B,T) est de nationalité G avec une vraisemblance de 0.24319759804841432


### MLE : approche sans l'utilisation de bayes naïf 

Comme nous allons estimer directement dans les données (par comptage, pas par produit), nous n'aurons pas besoin des fonctions de MAP que nous avons fait plus haut 

In [88]:
# Le code pour répondre à la question avec la deuxième approche est le suivant

def mle_sans_naif(data, cheveux, hauteur):
    classes = set(indiv[2] for ligne in data for indiv in ligne)
    vraisemblances = {}
    
    for c in classes:
        nb_c = sum(indiv[2] == c for ligne in data for indiv in ligne)
        nb_obs_c = sum(indiv[2] == c and indiv[0] == cheveux and indiv[1] == hauteur 
                       for ligne in data for indiv in ligne)
        
        vraisemblances[c] = nb_obs_c / nb_c if nb_c > 0 else 0
    
    best_class = max(vraisemblances, key=vraisemblances.get)
    return best_class, vraisemblances


In [89]:
# Application de la fonction avec le cas (B,T,G)

print(f"La distribution des proba entre les classes est: {mle_sans_naif(data=data, cheveux='B', hauteur='T')[1]} \nLa nationalité finale est: {mle_sans_naif(data=data, cheveux='B', hauteur='T')[0]}")

La distribution des proba entre les classes est: {'G': 0.22602739726027396, 'P': 0.2727272727272727} 
La nationalité finale est: P


## E- Point de vue continu

Dans ce cadre, on suppose que la mesure de la couleur des cheveux n’est plus binaire (B ou D) mais une variable continue notée intensité appartenant à [0,1] :

- 0 = blond pur
- 1 = dark pur
- Entre les deux, ce sont des valeurs intermédiaires.

1. Génération du jeu de données

Nous avons engender ce nouveau jeu de données avec la fonction qui génère aléatoirement des triplets (intensité, hauteur, pays)

In [78]:
import random

def generer_matrice_continu(n_lignes, n_colonnes):
    matrice = []
    for _ in range(n_lignes):
        ligne = []
        for _ in range(n_colonnes):
            intensite = round(random.uniform(0, 1), 5)  # valeur continue arrondie
            hauteur = random.choice(['T', 'S'])
            pays = random.choice(['G', 'P'])
            ligne.append((intensite, hauteur, pays))
        matrice.append(ligne)
    return matrice

# créons la matrice 10x10 qui contiendra 100 données 
random.seed(42)
data_continu = generer_matrice_continu(10, 10)

# Affichage
for ligne in data_continu:
    print(ligne)

[(0.63943, 'T', 'P'), (0.24489, 'T', 'G'), (0.6767, 'T', 'P'), (0.03178, 'T', 'G'), (0.23266, 'T', 'G'), (0.71602, 'S', 'G'), (0.44921, 'S', 'G'), (0.75881, 'T', 'P'), (0.34025, 'T', 'G'), (0.95721, 'S', 'G')]
[(0.09275, 'T', 'P'), (0.84749, 'S', 'G'), (0.72973, 'T', 'P'), (0.0788, 'S', 'P'), (0.57735, 'T', 'G'), (0.66126, 'S', 'G'), (0.85532, 'T', 'P'), (0.27797, 'S', 'G'), (0.37018, 'T', 'P'), (0.70182, 'T', 'G')]
[(0.53414, 'T', 'G'), (0.46226, 'S', 'G'), (0.68461, 'T', 'G'), (0.8218, 'S', 'P'), (0.26774, 'T', 'P'), (0.21263, 'S', 'P'), (0.88468, 'S', 'G'), (0.26488, 'T', 'P'), (0.74701, 'S', 'P'), (0.362, 'T', 'P')]
[(0.09091, 'T', 'G'), (0.15284, 'T', 'P'), (0.59641, 'S', 'P'), (0.59589, 'S', 'P'), (0.97108, 'T', 'G'), (0.68171, 'S', 'P'), (0.11155, 'S', 'G'), (0.45372, 'S', 'G'), (0.50768, 'T', 'P'), (0.84167, 'T', 'G')]
[(0.37391, 'T', 'G'), (0.59894, 'S', 'G'), (0.11187, 'S', 'P'), (0.23945, 'T', 'G'), (0.08565, 'S', 'G'), (0.97798, 'T', 'G'), (0.65975, 'T', 'P'), (0.52767, 'S'

2. Explication 

Dans ce contexte, la couleur des cheveux devient une variable continue, notée x, représentant une intensité comprise entre 0 et 1.
La formule de la vraisemblance s’écrit alors :

P(x,hauteur∣pays)=f(x∣pays)×P(hauteur∣pays)

où :
f(x∣pays) est la densité de probabilité de la variable continue x, que nous allons modéliser par une loi normale (gaussienne),

P(hauteur∣pays) reste une probabilité discrète estimée comme une fréquence, comme dans les questions précédentes.

Le principe du Maximum de Vraisemblance (MLE) consistera toujours à choisir le pays qui maximise cette vraisemblance P(x,hauteur∣pays).

Pour le cas du Maximum a posteriori (MAP), on multipliera en plus par la probabilité a priori du pays P(pays), comme dans la première partie.

Donc voici en gros comment nous allons fonctionner 


3. Modifications dans les fonctions Python

Dans les fonctions précédentes, on calculait P(B/G) avec la fonction pbg en comptant le nombre de personnes de hauteur G avec la couleur des cheveux Blond divisé par le nombre personne de hauteur G. Mais ici on va devoir remplacer cela par la fonction de densité gausienne f(x/G).

On pourra estimer les moyennes et l'écart-type des intensités de cheveux des individus de nationalité G par le MLE 

In [80]:
## On remplace pbg par cette version dans le cas continu
import math

def pbg_continu(data, x):
    # On extrait les intensités de cheveux pour la nationalité G
    cheveux_G = [indiv[0] for ligne in data for indiv in ligne if indiv[2] == 'G']
    if not cheveux_G:
        return 0
    
    mu_G = sum(cheveux_G) / len(cheveux_G)
    sigma_G = math.sqrt(sum((c - mu_G)**2 for c in cheveux_G) / len(cheveux_G))
    
    # Densité gaussienne évaluée en x
    return (1 / (math.sqrt(2 * math.pi) * sigma_G)) * math.exp(-((x - mu_G)**2) / (2 * sigma_G**2))

# On fait pareil pour pbp 



def pbp_continu(data, x):
    # On extrait les intensités de cheveux pour la nationalité P
    cheveux_P = [indiv[0] for ligne in data for indiv in ligne if indiv[2] == 'P']
    if not cheveux_P:
        return 0
    
    mu_P = sum(cheveux_P) / len(cheveux_P)
    sigma_P = math.sqrt(sum((c - mu_P)**2 for c in cheveux_P) / len(cheveux_P))
    
    # Densité gaussienne évaluée en x
    return (1 / (math.sqrt(2 * math.pi) * sigma_P)) * math.exp(-((x - mu_P)**2) / (2 * sigma_P**2))

# Nous allons utiliser ptg() et ptp() que nous avons définie dans la première partie 

# Maintenant avec ces fonctions, on va simplement remplacer le calcul de p1 et p2 qu'on faisait comme ceci:
#p1 = pbg(data) * ptg(data)
#p2 = pbp(data) * ptp(data)

# Par ceci: 
#p1 = pbg_continu(data, x) * ptg(data)
#p2 = pbp_continu(data, x) * ptp(data)

# Il faut remplacer x par une valeur continue quelconque qu'on aimerait prendre pour prédire la nationalité 

# Application des fonctions et prédiction de la nationalité 

# Nous allons prendre la première valeur de data_continu
x = data_continu[0][0][0]

# Calcul des proba 
p1 = pbg_continu(data_continu, x) * ptg(data_continu)
p2 = pbp_continu(data_continu, x) * ptp(data_continu)

# Affichage de la nationalité 
if p1 > p2:
    print(f"L'individu est de la nationalité G avec une vraisemblance de {p1}")
else:
    print(f"L'individu est de la nationalité P avec une vraisemblance de {p2}")


L'individu est de la nationalité P avec une vraisemblance de 0.6950617016358651


### Le calcul du MLE que nous avons fait prenait en compte l'utilisation de bayes naif

Maintenant nous aimerons bien calculer P(x,T/G) ou P(x,T/P) sans dissocier comme nous permet bayes naif à l'aide de l'hypothèse d'indépendance entre les données

Nous allons faire une fonction qui prend en paramètre le pays aussi afin de calculer les deux proba et comparer 

In [ ]:
# Il s'agit donc de calculer ici les occurences d'individus qui ont ces caractéristiques (x,T) simulatanément 

def p_joint_continu(data, x, taille, pays, tol=0.05):
    # tol : tolérance autour de x pour "approximer" un intervalle continu

    individus_classe = []
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == pays:
                individus_classe.append(indiv)

    if not individus_classe:
        return 0

    # Comptage des individus proches de x et de taille donnée
    nb_total = len(individus_classe)
    nb_voisins = sum(
        1 for indiv in individus_classe
        if abs(indiv[0] - x) < tol and indiv[1] == taille
    )

    return nb_voisins / nb_total

# Application du calcul 
p1 = p_joint_continu(data_continu, x, 'T', 'G')
p2 = p_joint_continu(data_continu, x, 'T', 'P')

if p1 > p2:
    print(f"L'individu est de la nationalité G avec une vraisemblance de {p1}")
else:
    print(f"L'individu est de la nationalité P avec une vraisemblance de {p2}")


L'individu est de la nationalité P avec une vraisemblance de 0.07142857142857142


### Commentaire
On constate bien que la proba ici est plus petite que la proba en utilisant bayes naif, ceci s'explique toujours par la rareté des évènements qui sont à la fois (x,T) dans ce jeu de données, supposer l'indépendance entre les données rend la solution plus stable 

### Application de ceci au calcul du MAP à présent 

Pour le MAP, il suffira d'ajouter la multiplication de P(pays) dans la formule P(x,T/pays) = f(x/pays).P(T/pays).P(pays)

Ainsi la proba pourra utiliser ce qu'on a calculé précédemment et ajouter juste P(pays)

In [ ]:
# Calcul des probabilités a priori
# Nous avions déjà calculé cela dans les parties précédentes P(pays) avec pays = 'G' / 'P' 
# pour rappel pg = p(pays=G) et pp = p(pays=P)

# Calcul MAP
p1_map = pbg_continu(data_continu, x) * ptg(data_continu) * pg(data_continu)
p2_map = pbp_continu(data_continu, x) * ptp(data_continu) * pp(data_continu)

if p1_map > p2_map:
    print(f"L'individu est probablement de nationalité G avec une probabilité proportionnelle à {p1_map}")
else:
    print(f"L'individu est probablement de nationalité P avec une probabilité proportionnelle à {p2_map}")


L'individu est probablement de nationalité G avec une probabilité proportionnelle à 0.39163171191109775
